In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [44]:
class Jackknife:
    def __init__( self, len_data, binsize ):
        self.binsize = binsize
        self.nbins = int(len_data/self.binsize)
        self.N = self.binsize * self.nbins
        self.jack_avg = []
        self.est = 0
        self.var_est = 0

    def set( self, func, list_of_data ):
        for i in range( self.nbins ):
            self.jack_avg.append( func( i, self.binsize, list_of_data ) )

    def do_it( self ):
        for i in range( 0, self.nbins ):
            self.est += self.jack_avg[i]
        self.est /= self.nbins

        for i in range( 0, self.nbins ):
            self.var_est += ( self.jack_avg[i] - self.est )**2
        self.var_est /= self.nbins
        self.var_est *= self.nbins -1

    def mean( self ):
        return self.est

    def var( self ):
        return self.var_est

    def err( self ):
        return np.sqrt(self.var_est)

def simple_mean(i, binsize, np_data):
    resmpld = np.delete(np_data, np.s_[i*binsize:(i+1)*binsize], axis=0)
    return np.mean(resmpld, axis=0)

def format_print(cen, err):
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'

def format_print_w_exact(exact, cen, err):
    for i in range(-50, 50):
        if 10**(-i+1)>=err>10**(-i):
            tmp=err*10**(i+1)
            return '{num:.{width}f}'.format(num=cen, width=i+1)+'('+str(round(tmp))+')'+' exact:'+'{ex:.{width}f}'.format(ex=exact, width=i+2)+' ['+'{num:.{width}f}'.format(num=abs(exact-cen)/err, width=2)+' sigma]'

In [45]:
def Z2x2(beta,h,kx,ky,kz):
    return 2.0*np.cosh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+8.0*np.cosh(2.0*h)+2.0*(np.exp(4.0*beta*(kx-ky-kz))+np.exp(4.0*beta*(-kx+ky-kz))+np.exp(4.0*beta*(-kx-ky+kz)))

def mag_per_vol2x2(beta,h,kx,ky,kz):
    numer = np.sinh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+2.0*np.sinh(2.0*h)
    denom = np.cosh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+4.0*np.cosh(2.0*h)+np.exp(4.0*beta*(kx-ky-kz))+np.exp(4.0*beta*(-kx+ky-kz))+np.exp(4.0*beta*(-kx-ky+kz))
    return numer/denom

def susc2x2(beta,h,kx,ky,kz):
    numer1 = 4.0*np.cosh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+4.0*np.cosh(2.0*h)
    numer2 = np.sinh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+2.0*np.sinh(2.0*h)
    denom = np.cosh(4.0*h)*np.exp(4.0*beta*(kx+ky+kz))+4.0*np.cosh(2.0*h)+np.exp(4.0*beta*(kx-ky-kz))+np.exp(4.0*beta*(-kx+ky-kz))+np.exp(4.0*beta*(-kx-ky+kz))
    return numer1/denom - 4.0*(numer2/denom)**2

# delta=1.0e-3
# print("mag")
# numeric=(np.log(Z2x2(beta, h+delta))-np.log(Z2x2(beta, h-delta)))/(2.0*delta)/4.0
# exact=mag_per_vol2x2(beta,h)
# print(format_print_w_exact(exact, numeric, delta**2))
# print("susc")
# numeric=(mag_per_vol2x2(beta,h+delta)-mag_per_vol2x2(beta,h-delta))/(2.0*delta)
# exact=susc2x2(beta,h)
# print(format_print_w_exact(exact, numeric, delta**2))

In [46]:
beta=0.294653
h=0.0
kx=0.4
ky=1.5
kz=1.1

In [72]:
nin=1000
nfin=10000
nint=1
binsize=10

In [73]:
mag_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/mag/"+str(n)+".dat")
    mag_.append(dat)
    
mag=np.array(mag_)

In [74]:
jk_mag = Jackknife(mag.size, binsize)
jk_mag.set(simple_mean, mag)
jk_mag.do_it()

In [75]:
print("mag_per_site")
format_print_w_exact(0.0, jk_mag.mean(), jk_mag.err())

mag_per_site


'-0.019(12) exact:0.0000 [1.57 sigma]'

In [76]:
jk_magsq = Jackknife(mag.size, binsize)
jk_magsq.set(simple_mean, mag**2/4)
jk_magsq.do_it()

In [77]:
print("susc")
exact=susc2x2(beta,h,kx,ky,kz)
format_print_w_exact(exact, jk_magsq.mean(), jk_magsq.err())

susc


'3.544(13) exact:3.5512 [0.58 sigma]'

In [78]:
ss_=[]

for n in range(nin,nfin,nint):
    dat=np.loadtxt("./obs/ss/"+str(n)+".dat")
    ss_.append(dat)
    
ss=np.array(ss_)

In [79]:
jk_ss = Jackknife(ss.size, binsize)
jk_ss.set(simple_mean, ss)
jk_ss.do_it()

In [80]:
jk_ss = Jackknife(ss.size, binsize)
jk_ss.set(simple_mean, ss)
jk_ss.do_it()

In [81]:
jk_ss.mean()

array([[1.        , 0.82933333],
       [0.87133333, 0.84311111]])

In [82]:
jk_ss.err()

array([[0.        , 0.00497706],
       [0.00415853, 0.00470909]])